In [45]:
# Find out number of threads
Threads.nthreads()

1

In [ ]:
## Distributed package # works
using Distributed
n = Int(1e8)

@time @sync nheads1 = @distributed (+) for i = 1:n; rand(Bool); end

nheads2 = 0;
@time for i = 1:n; global nheads2+=rand(Bool); end
println(nheads2)

## Show what worker that is involved
n = 32
a = zeros(n);
Threads.@threads for i = 1:n
    a[i] = Threads.threadid()
end
println(a)


# Distribute workers within for loop (Experimental only!)

n = Int(1e8)

sum = 0
@time Threads.@threads for i = 1:n
    global sum
    sum+=rand(Bool)
end
println(sum)

sum = 0
@time for i = 1:n
    global sum
    sum+=rand(Bool)
end
println(sum)

## Trying to map out distribution

In [ ]:
using Distributed
addprocs(1)
rmprocs(workers())

n_proc = 16
addprocs(n_proc); # add number of worker processes
@everywhere basepath = "/home/dreuter/Github/julia-paths"
@everywhere thispath = "/LearningPath-Julia/"
@everywhere fullpath = string(basepath,thispath)
@everywhere include(string(fullpath,"Counter.jl"))
Counter.count_heads(10)

In [22]:
function parallel_map(n,n_par)
    joblist = []
    n_job = Int(round(n/n_par))
    for i = 1:n_par
        push!(joblist, @spawn(Counter.count_heads(n_job)))
    end
    return joblist
end

parallel_map (generic function with 1 method)

In [40]:
n_par = 8;
n = 2e10;

## Compare with and without parallell jobs

In [44]:
joblist = parallel_map(n,n_par);
@time sum(map(fetch, joblist))

  7.691031 seconds (862 allocations: 29.844 KiB)


9999999263

In [ ]:
@time Counter.count_heads(n)